In [1]:
import os
import pydot
import torch
from sklearn.model_selection import train_test_split
import time

In [2]:
import glob
import os
import numpy as np
import pickle
import re

labels = []
sentences = []
for file in glob.glob('./txt/*.txt', recursive=True):
    header =""
    print(file)
    with open(file, 'r') as f:
        count = 0
        sents = re.split(r'\.\s', f.read())
        for sentence in sents:
            if "Краткая информация" in sentence:
                header = 0
            elif "Диагностика" in sentence:
                header = 1
            elif "Лечение" in sentence:
                header = 2
            elif "Реабилитация" in sentence:
                header = 3
            elif "Профилактика" in sentetnce:
                header = 4
            if header != '':
                labels.append(header)
                sentences.append(sentence.replace('\n', ' '))
       

./txt/КР472.txt
./txt/КР8.txt
./txt/КР116.txt
./txt/КР77.txt
./txt/КР76.txt
./txt/КР62.txt
./txt/КР329.txt
./txt/КР467.txt
./txt/КР249.txt
./txt/КР507.txt
./txt/КР277.txt
./txt/КР288.txt
./txt/КР317.txt
./txt/КР129.txt
./txt/КР48.txt
./txt/КР75.txt
./txt/КР289.txt
./txt/КР504.txt
./txt/КР266.txt
./txt/КР460.txt
./txt/КР138.txt
./txt/КР71.txt
./txt/КР58.txt
./txt/КР70.txt
./txt/КР111.txt
./txt/КР139.txt
./txt/КР475.txt
./txt/КР515.txt
./txt/КР501.txt
./txt/КР529.txt
./txt/КР339.txt
./txt/КР66.txt
./txt/КР73.txt
./txt/КР67.txt
./txt/КР98.txt
./txt/КР476.txt
./txt/КР338.txt
./txt/КР502.txt
./txt/КР565.txt
./txt/КР607.txt
./txt/КР161.txt
./txt/КР613.txt
./txt/КР149.txt
./txt/КР612.txt
./txt/КР160.txt
./txt/КР610.txt
./txt/КР16.txt
./txt/КР611.txt
./txt/КР588.txt
./txt/КР173.txt
./txt/КР601.txt
./txt/КР12.txt
./txt/КР614.txt
./txt/КР589.txt
./txt/КР574.txt
./txt/КР400.txt
./txt/КР372.txt
./txt/КР164.txt
./txt/КР602.txt
./txt/КР11.txt
./txt/КР39.txt
./txt/КР10.txt
./txt/КР603.txt
./txt/КР561

'Сравнительно редко встречаются огнестрельные переломы  В  настоящее  время  возросла  частота  транспортных  и бытовых травм [56, 77, 92]'

In [13]:
X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.5)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [4]:
import pandas as pd
data = pd.DataFrame()
data['id'] = [i for i in range(len(y_train))]
data['label'] = y_train
data['text'] = X_train

In [14]:


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [3]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('DeepPavlov/rubert-base-cased-sentence', model_max_length=512)

In [4]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
val_encodings = tokenizer(X_test, truncation=True, padding=True)

NameError: name 'X_train' is not defined

In [9]:
print(train_encodings[0])

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [17]:
train_dataset = MyDataset(train_encodings, y_train)
val_dataset = MyDataset(val_encodings, y_test)

In [20]:
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased-sentence', num_labels=4)
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)


    running_loss = 0
    total_train_loss = 0
    start_time = time.time()

    for i,batch in enumerate(train_loader):   
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()
        
        running_loss += loss.data
        total_train_loss += loss.data

        print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                epoch+1, int(100 * (i+1) / 16), running_loss, time.time() - start_time))
        running_loss = 0.0
        start_time = time.time()

model.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, 6% 	 train_loss: 1.36 took: 65.99s
Epoch 1, 12% 	 train_loss: 1.22 took: 64.24s
Epoch 1, 18% 	 train_loss: 1.26 took: 65.64s
Epoch 1, 25% 	 train_loss: 1.34 took: 66.77s
Epoch 1, 31% 	 train_loss: 1.76 took: 68.53s
Epoch 1, 37% 	 train_loss: 1.47 took: 68.23s
Epoch 1, 43% 	 train_loss: 1.23 took: 67.01s
Epoch 1, 50% 	 train_loss: 1.30 took: 69.01s
Epoch 1, 56% 	 train_loss: 1.37 took: 68.28s
Epoch 1, 62% 	 train_loss: 1.17 took: 65.99s
Epoch 1, 68% 	 train_loss: 1.38 took: 65.11s
Epoch 1, 75% 	 train_loss: 1.29 took: 64.94s
Epoch 1, 81% 	 train_loss: 1.19 took: 67.29s
Epoch 1, 87% 	 train_loss: 1.23 took: 65.84s
Epoch 1, 93% 	 train_loss: 1.20 took: 65.89s
Epoch 1, 100% 	 train_loss: 1.22 took: 64.52s
Epoch 1, 106% 	 train_loss: 1.10 took: 63.12s
Epoch 1, 112% 	 train_loss: 1.17 took: 68.36s
Epoch 1, 118% 	 train_loss: 1.40 took: 65.62s
Epoch 1, 125% 	 train_loss: 1.18 took: 64.72s
Epoch 1, 131% 	 train_loss: 1.29 took: 62.89s
Epoch 1, 137% 	 train_loss: 1.03 took: 62.15s
Epoc

Epoch 1, 1118% 	 train_loss: 0.99 took: 64.81s
Epoch 1, 1125% 	 train_loss: 0.79 took: 64.43s
Epoch 1, 1131% 	 train_loss: 1.22 took: 64.06s
Epoch 1, 1137% 	 train_loss: 0.57 took: 64.11s
Epoch 1, 1143% 	 train_loss: 1.28 took: 64.61s
Epoch 1, 1150% 	 train_loss: 1.03 took: 64.95s
Epoch 1, 1156% 	 train_loss: 1.14 took: 64.38s
Epoch 1, 1162% 	 train_loss: 0.98 took: 64.28s
Epoch 1, 1168% 	 train_loss: 1.21 took: 64.11s
Epoch 1, 1175% 	 train_loss: 0.81 took: 63.88s
Epoch 1, 1181% 	 train_loss: 1.09 took: 62.83s
Epoch 1, 1187% 	 train_loss: 0.91 took: 64.18s
Epoch 1, 1193% 	 train_loss: 0.91 took: 64.37s
Epoch 1, 1200% 	 train_loss: 0.44 took: 64.36s
Epoch 1, 1206% 	 train_loss: 0.78 took: 64.08s
Epoch 1, 1212% 	 train_loss: 0.89 took: 63.85s
Epoch 1, 1218% 	 train_loss: 1.22 took: 64.23s
Epoch 1, 1225% 	 train_loss: 1.04 took: 63.68s
Epoch 1, 1231% 	 train_loss: 0.90 took: 64.63s
Epoch 1, 1237% 	 train_loss: 1.16 took: 63.86s
Epoch 1, 1243% 	 train_loss: 1.00 took: 63.67s
Epoch 1, 1250

KeyboardInterrupt: 

In [24]:
torch.save(model, "my.model")

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = torch.load("../model-final2", map_location=device)
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [19]:
from torch.utils.data import DataLoader
import numpy
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)
model.eval()

y = numpy.array([])
y_pred = numpy.array([])
with torch.no_grad():
    for i, batch in enumerate(val_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        y = numpy.append(y, labels.cpu().detach().numpy())
        y_pred = numpy.append(y_pred, numpy.argmax(outputs[0].cpu().detach().numpy(), 1))

print(accuracy_score(y, y_pred))
        
        

KeyboardInterrupt: 

In [18]:
embeds = []
for s in sentences:
    tokens = tokenizer([s], truncation=True, padding=True)
    input_ids = torch.tensor(tokens.input_ids).to(device)
    att = torch.tensor(tokens.attention_mask).to(device)
    outputs = model(input_ids, attention_mask=att, output_hidden_states=True)
    embeds.append(outputs.hidden_states[0])


KeyboardInterrupt: 

tuple

In [13]:
import numpy as np
encoding = None
while True:
    request = input()
    encoding = tokenizer([request], truncation=True, padding=True)
    input_ids = torch.tensor(encoding.input_ids).to(device)
    att = torch.tensor(encoding.attention_mask).to(device)
    outputs = model(input_ids, attention_mask=att)
    label = np.argmax(outputs[0].detach().numpy(), 1)[0]
    required_sentences = [y for x,y in list(zip(labels, sentences)) if x == label]
    required_encodings = tokenizer(required_sentences, truncation=True, padding=True)
    distances = []

sjdksjk


KeyboardInterrupt: Interrupted by user

In [14]:
encoding

{'input_ids': [[101, 268, 251, 239, 12188, 251, 253, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1]]}

In [23]:
from sklearn.metrics import classification_report
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

         0.0       0.82      0.88      0.85      4568
         1.0       0.85      0.75      0.79      6458
         2.0       0.87      0.86      0.87     11303
         3.0       0.74      0.81      0.77      4743

    accuracy                           0.83     27072
   macro avg       0.82      0.83      0.82     27072
weighted avg       0.83      0.83      0.83     27072

